<a href="https://colab.research.google.com/github/hyeonu96/first-repository/blob/main/%5BEX_13%5D%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_Movielens_%EC%98%81%ED%99%94_%EC%B6%94%EC%B2%9C_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트 - Movielens 영화 추천 실습

## 1) 데이터 준비와 전처리

In [ ]:
import numpy as np
import scipy
import implicit
import pandas as pd

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


## 2) 분석해 봅시다.
ratings에 있는 유니크한 영화 개수
ratings에 있는 유니크한 사용자 수
 가장 인기 있는 영화 30개(인기순)

In [ ]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [ ]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head(15)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 3114: invalid continuation byte

In [ ]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [ ]:
#rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [ ]:
# movies와 ratings를 합쳐준다.
data= pd.merge(movies, ratings)

In [ ]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = [ 'user_id','title', 'rating']
data = data[using_cols]
data.head(10)

,user_id,title,rating
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
5,18,Toy Story (1995),4
6,19,Toy Story (1995),5
7,21,Toy Story (1995),3
8,23,Toy Story (1995),4
9,26,Toy Story (1995),3


In [ ]:
data['title'] = data['title'].str.lower() 

In [ ]:
#가장 인기있는 영화 30개(인기순)
movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [ ]:
# 유저별 몇 편의 영화를 보고 있는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [ ]:
# 내가 좋아하시는 영화의 데이터로 바꿔서 추가한다.
my_favorite = ['men in black (1997)' , 'fugitive, the (1993)' ,'terminator, the (1984)' ,
               'back to the future (1985)' ,'forrest gump (1994)']

# 'kimin'이라는 user_id가 위 영화의 평점을 5점씩 주었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['kimin']*5, 'title': my_favorite, 'rating':[5]*5})

# user_id에 'kimin'이라는 데이터가 없다면
# 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
if not data.isin({'user_id':['kimin']})['user_id'].any():  
    data = data.append(my_playlist)   
    
data.tail(20) 

,user_id,title,rating
836463,5074,"contender, the (2000)",4
836464,5087,"contender, the (2000)",4
836465,5100,"contender, the (2000)",4
836466,5205,"contender, the (2000)",4
836467,5304,"contender, the (2000)",4
836468,5333,"contender, the (2000)",4
836469,5359,"contender, the (2000)",5
836470,5405,"contender, the (2000)",4
836471,5475,"contender, the (2000)",5
836472,5602,"contender, the (2000)",3


In [ ]:
모델에 활용하기 위한 전처리

In [ ]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 타이틀 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [ ]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['kimin'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(title_to_idx['forrest gump (1994)'])

6039
342


In [ ]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,rating
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,1419,5
1,6039,439,5
2,6039,1122,5
3,6039,1152,5


## 4) CSR matrix를 직접 만들어 봅시다.

In [ ]:
data['rating']

0    5
1    4
2    4
3    5
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: rating, Length: 836483, dtype: int64

In [ ]:
print(data.rating)

0    5
1    4
2    4
3    5
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: rating, Length: 836483, dtype: int64


In [ ]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.rating, (data.user_id, data.title)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [ ]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [ ]:
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
kimin, back_to_the_future = user_to_idx['kimin'], title_to_idx['back to the future (1985)']
kimin_vector, back_to_the_future_vector = als_model.user_factors[kimin], als_model.item_factors[back_to_the_future]

In [ ]:
kimin_vector

array([ 0.7619857 ,  0.3158938 ,  0.38358492,  0.25647193,  0.11597677,
        0.02117402,  0.16915922, -0.17889501, -0.5150856 ,  0.39830554,
        0.10664093,  0.5155381 ,  0.19273898,  0.0180732 , -0.4412216 ,
        0.31204912,  0.17514154,  0.59714854,  0.01702799, -0.62830997,
       -0.01207074, -0.52381027, -0.01553841,  0.39696553,  0.16513838,
        0.01061953,  0.45297828,  0.48572776,  0.37537232,  0.11107843,
       -0.01496267, -0.5255054 ,  0.20902935,  0.26180694, -0.07782665,
       -0.76092666, -0.43672633,  0.06373069,  0.7163566 ,  0.37947428,
       -0.20390718,  0.9472401 , -0.48845   , -1.0109347 ,  0.13481432,
        0.42775318, -0.0060963 ,  0.06903312,  0.7712032 , -0.44617388,
        0.28635725,  0.24020046,  0.4900563 , -0.83173794, -0.59908897,
        0.20332296,  0.2911755 ,  0.5319935 ,  0.9252862 , -0.4278859 ,
       -0.6041462 ,  0.0415399 , -0.08341131, -0.02205225, -0.3336285 ,
        0.1339113 , -0.17611422, -0.02984286,  0.5074986 ,  0.71

In [ ]:
back_to_the_future_vector

array([-1.10074850e-02,  4.28676140e-03,  2.91329678e-02, -8.11146945e-03,
        8.85589141e-03,  2.88015697e-02,  3.03968638e-02,  4.38258015e-02,
       -2.14026608e-02,  3.16830352e-02,  1.06804902e-02,  2.32292749e-02,
        5.31970290e-03,  2.51689041e-03, -2.77814399e-02,  1.60127450e-02,
        5.64748654e-03,  3.59635279e-02, -1.90089887e-03, -3.09873503e-02,
        8.09556805e-03, -2.12735875e-04, -1.04071666e-02,  2.59525590e-02,
        2.52082981e-02, -2.86593847e-03, -2.51530111e-02, -1.53997503e-02,
        1.44338217e-02,  3.85816321e-02,  8.80394503e-03, -2.41759941e-02,
        5.12862578e-02,  2.27679443e-02, -2.75468361e-02, -2.33357530e-02,
       -1.48051577e-02,  1.81689188e-02,  2.26280913e-02,  3.31120454e-02,
        3.53832613e-03,  3.15410718e-02,  8.99394043e-03, -2.83874255e-02,
        1.30236186e-02,  4.30456027e-02,  1.81171689e-02, -1.09882036e-03,
        5.29957525e-02, -8.78426526e-03,  4.70098481e-02,  2.20345724e-02,
        5.34342006e-02, -

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [ ]:
np.dot(kimin_vector, back_to_the_future_vector)

0.60882354

In [ ]:
toy = title_to_idx['toy story (1995)']
toy_vector = als_model.item_factors[toy]
np.dot(kimin_vector, toy_vector)

0.22426032

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [ ]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [ ]:
get_similar_title('men in black (1997)') 

['men in black (1997)',
 'jurassic park (1993)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'independence day (id4) (1996)',
 'matrix, the (1999)',
 'fifth element, the (1997)',
 'lost world: jurassic park, the (1997)',
 'face/off (1997)',
 'true lies (1994)']

In [ ]:
get_similar_title('toy story (1995)') 

['toy story (1995)',
 'toy story 2 (1999)',
 'babe (1995)',
 "bug's life, a (1998)",
 'aladdin (1992)',
 'groundhog day (1993)',
 'lion king, the (1994)',
 'pleasantville (1998)',
 'beauty and the beast (1991)',
 'forrest gump (1994)']

In [ ]:
get_similar_title('back to the future (1985)') 

['back to the future (1985)',
 "ferris bueller's day off (1986)",
 'back to the future part ii (1989)',
 'when harry met sally... (1989)',
 'big (1988)',
 'ghostbusters (1984)',
 'bull durham (1988)',
 'fish called wanda, a (1988)',
 'airplane! (1980)',
 'cocoon (1985)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [ ]:
user = user_to_idx['kimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
for i in title_recommended: 
    print("{} {:08.3f}".format(idx_to_title[i[0]], i[1]))

terminator 2: judgment day (1991) 0000.631
jurassic park (1993) 0000.555
matrix, the (1999) 0000.541
total recall (1990) 0000.429
sixth sense, the (1999) 0000.382
die hard (1988) 0000.349
star wars: episode vi - return of the jedi (1983) 0000.342
braveheart (1995) 0000.330
groundhog day (1993) 0000.324
hunt for red october, the (1990) 0000.312
star wars: episode iv - a new hope (1977) 0000.281
star wars: episode v - the empire strikes back (1980) 0000.280
saving private ryan (1998) 0000.275
silence of the lambs, the (1991) 0000.262
speed (1994) 0000.259
airplane! (1980) 0000.245
face/off (1997) 0000.245
independence day (id4) (1996) 0000.237
e.t. the extra-terrestrial (1982) 0000.226
aliens (1986) 0000.225


In [ ]:
rihanna = title_to_idx['die hard (1988)']
explain = als_model.explain(user, csr_data, itemid=rihanna)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('terminator, the (1984)', 0.21345599656235403),
 ('fugitive, the (1993)', 0.16121396610182404),
 ('back to the future (1985)', 0.04354122396577952),
 ('forrest gump (1994)', -0.0069389088459887805),
 ('men in black (1997)', -0.06626752197814771)]

루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

평가문항	상세기준
1. CSR matrix가 정상적으로 만들어졌다.

사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.

사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.

MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정하고 의미를 분석해보았다.

회고

요즘 굉장히 핫한 아이템인 추천, 알고리즘 시스템을 알아보았는데 왜 핫하고 뜨고 있는지 알게 되었다.
 우선 비슷한 종류를 찾아내는 것이 신기함.
  비슷한 종류를 찾아내어 새로운 비슷한 것들까지 도달하는 것이 새로웠다.

  하지만 정확성 관련해서는 아직 부족한 부분도 있는거 같다.
  정확성을 높일 수 있는 방법을 알아보자.
  